##### Binary encoding

In [1]:
def bin_enc(lbl):
    mi = min(lbl)
    length = len(bin(max(lbl)-mi+1)[2:])
    enc=[]
    for i in lbl:
        b=bin(i-mi)[2:].zfill(length)
        enc.append([int(n) for n in b])
        return enc

def bin_dec(enc, mi=0):
    lbl=[]
    for e in enc:
        rounded=[int(round(x)) for x in e] 
        string= ''.join(str(x) for x in rounded)
        num=int(string,2) + mi
        lbl.append(num)
    return lbl

##### One-hot encoding

In [2]:
def onehot_enc(lbl, min_val=0):
    mi=min(lbl)
    enc=np.full((len(lbl),max(lbl)-mi+1), min_val, np.int8)
    
    for i, x in enumerate(lbl):
        enc[i, x-mi]=1
    return enc

def onehot_dec(enc, mi=0):
    return [np.argmax(e)+mi for e in enc]

##### Fungsi aktivasi dan turunannya

In [3]:
# Fungsi aktivasi sigmoid dan turunannya
def sig(X):
    return [1/(1+np.exp(-x)) for x in X]

def sigd(X):
    output=[]
    for i, x in enumerate(X):
        s = sig([x])[0]
        output.append(s*(1-s))
    return output

##### Fungsi modeling (training) backpropagation

In [4]:
def bp_fit(X, target, layer_conf, max_epoch, max_error=.1, learn_rate=.1, 
           print_per_epoch=100): 
    nin=[np.empty(i) for i in layer_conf]
    n = [np.empty(j+1) if i<len(layer_conf)-1 
         else np.empty(j) for i, j in enumerate(layer_conf)]
    w = np.array([np.random.rand(layer_conf[i]+1, layer_conf[i+1])
        for i in range(len(layer_conf)-1)]) 
    dw = [np.empty((layer_conf[i]+1, layer_conf[i+1])) 
        for i in range(len(layer_conf)-1)] 
    d = [np.empty(s) for s in layer_conf[1:]] 
    din = [np.empty(s) for s in layer_conf[1:-1]] 
    epoch = 0 
    mse = 1
    for i in range(0, len(n)-1): 
        n[i][-1]=1 
    while (max_epoch == -1 or epoch<max_epoch) and mse>max_error:
        epoch +=1 
        mse = 0 
        
        for r in range(len(X)): 
            n[0][:-1]=X[r]
            
            for L in range(1, len(layer_conf)):
                nin[L] = np.dot(n[L-1], w[L-1]) 
                n[L][:len(nin[L])]=sig(nin[L]) 
                
            e = target[r] - n[-1] 
            mse += sum(e ** 2) 
            d[-1]=e*sigd(nin[-1]) 
            dw[-1]=learn_rate * d[-1]*n[-2].reshape((-1,1)) 
            
            for L in range(len(layer_conf)-1, 1, -1): 
                din[L-2]=np.dot(d[L-1], np.transpose(w[L-1][:-1])) 
                d[L-2]=din[L-2]*np.array(sigd(nin[L-1])) 
                dw[L-2]=(learn_rate*d[L-2])*n[L-2].reshape((-1,1)) 
            w += dw 
        mse /= len(X) 
        if print_per_epoch > -1 and epoch % print_per_epoch == 0: 
            print(f'Epoch {epoch}, MSE: {mse}') 
        return w, epoch, mse

##### Fungsi pengujian backpropagation

In [5]:
def bp_predict(X,w): 
    n=[np.empty(len(i)) for i in w] 
    nin=[np.empty(len(i[0])) for i in w] 
    predict = [] 
    
    n.append(np.empty(len(w[-1][0]))) 
    
    for x in X: 
        n[0][:-1]=x 
        
        for L in range(0, len(w)):
            nin[L] = np.dot(n[L], w[L]) 
            n[L+1][:len(nin[L])] = sig(nin[L]) 
        predict.append(n[-1].copy()) 
    return predict

##### Import library

In [6]:
# !pip install neurolab

In [7]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import minmax_scale
from sklearn.metrics import accuracy_score
import numpy as np

##### Load data, pisahkan data dan label. Konversi label dengan one-hot encoding

In [8]:
iris = datasets.load_iris()
X = minmax_scale(iris.data)
Y = onehot_enc(iris.target)

##### Preprocess data: membagi data training, label training dan data testing, label testing (biasanya 2/3 data training, 1/3 data testing)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=.3)

##### Arsitektur jaringan
Input: 4 atribut, output: [iris setosa/iris versicolor/iris virginica] atau dengan one-hot encoding menjadi kelas [1 0 0], [0 1 0], [0 0 1]
Back-propagation dengan 1 hidden layer. Hidden layer mempergunakan 4 neuron (bisa divariasi). Oleh karena itu arsitektur jaringan berbentuk:
4 – 4 – 3 (4 neuron input – 4 neuron di hidden layer – 3 neuron output)

##### Membentuk model JST menggunakan data training dan label training

In [10]:
w, ep, mse = bp_fit(X_train, y_train, layer_conf=(4,3,3), learn_rate=.1, max_epoch=1000, max_error=.1, print_per_epoch=25)
print(f'Epochs: {ep}, MSE: {mse}')

Epochs: 1, MSE: 0.9456097492821648


C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\ipykernel_launcher.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  import sys
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\ipykernel_launcher.py:36: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


##### Menguji model

In [11]:
predict = bp_predict(X_test, w)
predict = onehot_dec(predict)

##### Membandingkan luaran dengan target

In [12]:
y_test = onehot_dec(y_test)
acc = accuracy_score(predict, y_test)
print(f'Output: {predict}')
print(f'True: {y_test}')
print(f'Accuracy: {acc}')

Output: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
True: [2, 2, 2, 0, 1, 0, 0, 1, 2, 0, 0, 1, 0, 1, 1, 2, 2, 1, 2, 2, 0, 2, 0, 1, 2, 0, 0, 2, 2, 2, 1, 1, 1, 2, 1, 2, 0, 1, 1, 0, 0, 0, 2, 0, 1]
Accuracy: 0.3111111111111111


##### Prediksi, identifikasi, klasifikasi data baru (tunggal)

In [13]:
irisBaru=[6.7, 3.3, 5.7 ,2.5]
identitas=bp_predict(irisBaru, w)
identitas=onehot_dec(identitas)

##### Menemukan model yang optimal:
Variasi jumlah neuron dalam hidden layer (10, 15, 20, 25, 30…) untuk menemukan akurasi tertinggi
− Normalisasi data (minmax, z-score)